In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
%%time
arrest_data = pd.read_csv('data/arrest-data-from-2010-to-present.csv')
crime_data = pd.read_csv('data/crime-data-from-2010-to-present.csv')
income_data = pd.read_csv('data/LAIncome.csv', sep='\t')
moon_data = pd.read_csv('data/full_moon.csv')
race_data = pd.read_csv('data/LARace.csv')
zip_data = pd.read_csv('data/ziplatlon.csv', dtype={'ZIP': 'str'})

CPU times: user 18.8 s, sys: 1.23 s, total: 20 s
Wall time: 20.3 s


In [3]:
arrest_data.columns

Index(['Report ID', 'Arrest Date', 'Time', 'Area ID', 'Area Name',
       'Reporting District', 'Age', 'Sex Code', 'Descent Code',
       'Charge Group Code', 'Charge Group Description', 'Arrest Type Code',
       'Charge', 'Charge Description', 'Address', 'Cross Street', 'Location',
       'Zip Codes', 'Census Tracts', 'Precinct Boundaries',
       'LA Specific Plans', 'Council Districts',
       'Neighborhood Councils (Certified)'],
      dtype='object')

In [4]:
income_data.head()

,Zip,Community,Amount
0,90001,"Los Angeles (South Los Angeles), Florence-Graham","$43,360"
1,90002,"Los Angeles (Southeast Los Angeles, Watts)","$37,285"
2,90003,"Los Angeles (South Los Angeles, Southeast Los ...","$40,598"
3,90004,"Los Angeles (Hancock Park, Rampart Village, Vi...","$49,675"
4,90005,"Los Angeles (Hancock Park, Koreatown, Wilshire...","$38,491"


In [5]:
moon_data.head()

,Day,Date,Time
0,Monday,15 January 1900,08:07:30 pm
1,Wednesday,14 February 1900,02:50:12 pm
2,Friday,16 March 1900,09:11:48 am
3,Sunday,15 April 1900,02:02:06 am
4,Monday,14 May 1900,04:36:36 pm


### Hypothesis 1: Area with the highest black population has the most arrests under vandalism 

In [25]:
eval(arrest_data['Location'][0])

{'latitude': '33.992',
 'human_address': '{"address": "", "city": "", "state": "", "zip": ""}',
 'longitude': '-118.4201'}

In [7]:
race_data['Zip Code'].nunique()

110

In [29]:
def convert_lat_lon(x):
    return (eval(x)['latitude'], eval(x)['longitude']) 
convert_lat_lon = np.vectorize(convert_lat_lon)

In [30]:
arrest_data['Lat'], arrest_data['Lon'] = convert_lat_lon(arrest_data['Location'])

In [42]:
zip_data = zip_data[zip_data['ZIP'].apply(lambda x: x[:2]=='90')]

In [49]:
sample_lat, sample_lon = arrest_data['Lat'][0], arrest_data['Lon'][0] 

In [84]:
temp = pd.DataFrame( { 'ZIP':zip_data['ZIP'], 'DifLat': zip_data['LAT'] - float(sample_lat) , 'DifLon': zip_data['LNG'] - float(sample_lon)})
temp['ToMin'] = temp['DifLat']**2 + temp['DifLon']**2
temp

,ZIP,DifLat,DifLon,ToMin
30021,90001,-0.017973,0.170591,0.029424
30022,90002,-0.042901,0.173363,0.031895
30023,90003,-0.027869,0.147317,0.022479
30024,90004,0.084198,0.109378,0.019053
30025,90005,0.067163,0.113208,0.017327
...,...,...,...,...
30187,90813,-0.209741,0.223307,0.093857
30188,90814,-0.220384,0.276469,0.125004
30189,90815,-0.197652,0.303709,0.131305
30190,90822,-0.213564,0.301452,0.136483


In [86]:
temp[temp['ToMin']==temp['ToMin'].min()]

,ZIP,DifLat,DifLon,ToMin
30077,90066,0.010028,-0.010556,0.000212


In [88]:
sample_lat, sample_lon

('33.992', '-118.4201')